## Todo:
- write calibration as py file with MPI
- re-write all evaluations of cross sections to include ELM Coulomb correction for protons
- regenerate prior to use CHUQ-like geometry with larger diffuseness
- use 10% model error term
- re-run el and qel calibrations
- uq for Rskin, Esym, L

## How to use these notbooks

These notebooks walk through the gathering of data to use as evidence, determination of a prior, calibration, and visualization of a global optical potential using JITR. 

The potential is the East Lansing Model (ELM), described in `elm.py` and described below.


The notebooks should be run in the following order:

0. run this file, `README.ipynb`, to install relevant packages
1. run `gather_evidence.ipynb` to gather data from exfor to calibrate to, and set up the workspaces we will use for calibration and visualization
2. run `generate_kduq_pred_post.ipynb` to see how well KDUQ does to describe this data
3. run `generate_ELM_prior.ipynb` to generate a reasonable prior based on KDUQ
4. run `run_ELM_cal.ipynb` to run the calibration
5. run `elm_predictive_post_xs.ipynb` to generate predictive posteriors

## What scientific software will we use?
The usual suspects (numpy, scip, matplotlib, etc.) will do a lot of the heavy lifting for us. We will also use some more specialized librarier:

- [jitr](https://github.com/beykyle/jitr) parametric reaction code
- [exfor-tools](https://github.com/beykyle/exfor_tools) using an [x4i3](https://github.com/afedynitch/x4i3/) backend for grabbing data from [EXFOR](https://www-nds.iaea.org/exfor/) (requires 1GB of disk space to download).
- [corner](https://corner.readthedocs.io/en/latest/) for nice plots

## Fitting philosophy
We would like to start with a more limited form and scale up to a more complicated one as demanded by the evidence. In particular, to begin, we will not consider analyzing powers or other polarization data, and instead freeze the spin-orbit contribution to that of previously fit potentials.

In [9]:
!pip install jitr
!pip install exfor-tools
!pip install corner

## the East Lansing Model

We would like to calibrate the East Lansing model. The East Lansing Model (ELM) is Lane consistent,

\begin{align}
    U(r;E,A,Z) &= U_0(r;E,A) \pm \alpha U_1(r;E,A) + V_C(r),
\end{align}

with the plus(minus) referring to neutrons(protons), and $\alpha = (N-Z)/(N+Z)$. The energy argument is the center of mass energy, with a Coulomb correction in the case of protons:

\begin{equation}
E = \begin{cases} E_{cm} & \rm{neutron} \\ E_{cm} - \frac{6 Z e^2}{5 R_C} & \rm{proton} \\ \end{cases}
\end{equation}


$V_C$ is the standard Coulomb charged sphere form (see [Koning Delaroche](https://doi.org/10.1016/S0375-9474(02)01321-0) Eq. 4):

\begin{align}
    V_C(r;Z) &= \begin{cases} \frac{Z z e^2}{2 R_c} \left( 3  - r^2/R_C^2\right) & r <= R_C \\ Zze^2/r &r > R_C \end{cases}.
\end{align}


The isoscalar $U_0$ and isovector $U_1$ components are free to have different geometries and energy dependencies. These are:

\begin{align}
    U_0(r,E,A) &= 
    -\bigg( V_0(E) + i W_0(E)  \bigg) f(r,R_0,a_0)  - 4 i a_0 W_{D0}(E) \frac{d}{dr} f(r,R_0,a_0) \\
    & + V_{so}  \bigg( \frac{\hbar}{m_\pi c}\bigg)^2 \frac{1}{r} \frac{d}{dr}  f(r,R_0,a_0),
\end{align}

and

\begin{align}
    U_1(r,E,A) &= 
    -\bigg( V_1(E) + i W_1(E)  \bigg) f(r,R_1,a_1)  - 4 i a_1 W_{D1}(E) \frac{d}{dr} f(r,R_1,a_1) \\
\end{align}


Here, $f(r,R,a)$ is the Woods-Saxon form factor. Note that all isoscalar terms shared the same geometry, likewiswe with isovector. Only the isoscalar part has a spin-orbit coupling. 

The radius of the isoscalar potential $R_0$ and the diffuseness $a_0$ are parameterized as without any $E$ or $A$ dependence aside from the typical volume term. The free parameters $r_0$, $a_0$, $r_1$ and $a_1$ are in units of fm. The radii that enter into the Woods-Saxon form factors are then

\begin{align}
    R_0 &= r_0 A^{1/3} \\
    R_1 &= r_1 A^{1/3}.
\end{align}

The Coulomb radius $R_C$ is determined by the isoscalar geometry:

\begin{equation}
R_C = R_0.
\end{equation}

One can approximately consider the isovector geometry as corresponding to the isovector skin of the nucleus; with $R_1 > R_0$ corresponding to the formation of a neutron skin for neutron-rich nuclei, and a proton skin for proton-rich nuclei. The strength of this isovector term is proportional to $\alpha$. 


The depths of each of are parameterized as a function of $E- E_F$, the difference between the center-of-mass frame energy between the projectile and target, and the Fermi energy of the target, the latter of which is not treated as a free parameter but is instead taken from experiment. The isoscalar depths terms are

\begin{align}
    V_0(E) &= V_0 \big( 1 + \alpha (E - E_f) + \beta (E- E_F)^2 \big) \\
    W_0(E) &= W_0 \frac{(E - E_f)^2}{(E-E_F)^2 + \gamma_W^2} \\
    W_{D0}(E) &= W_{D0}  \frac{(E - E_f)^2}{(E-E_F)^2 + \gamma_D^2} \, \exp{\left( -\frac{E-E_f}{\gamma_D}  \right)} \\
    V_{so}(E) &= V_{so}   \\
\end{align}

Note the neglect of any $A$ dependence of the depths aside from the isovector factors. This was chosen due to recent uncertainty quantified calibration of global optical potentials to elastic scattering observables indicating that the correction terms linear in $A$ typically are at least an order of magnitude smaller than corresponding $A$ independent terms, and have uncertainties on the order of their magnitudes. The isovector depths are

\begin{align}
    V_1(E) &= V_1 \big( 1 + \alpha (E - E_f) + \beta (E- E_F)^2 \big) \\
    W_1(E) &= W_1 \frac{(E - E_f)^2}{(E-E_F)^2 + \gamma_W^2} \\
    W_{D1}(E) &= W_{D1}  \frac{(E - E_f)^2}{(E-E_F)^2 + \gamma_D^2} \, \exp{\left( -\frac{E-E_f}{\gamma_D}  \right)} \\
\end{align}

Notice that the isovector and isoscalar terms share energy dependencies. In total, the depths have 11 free parameters; 7 magnitudes $V_0$, $V_1$, $W_0$, $W_1$ $W_{D0}$, $W_{D1}$ $V_{so}$, 2 terms for the polynomial energy dependence of $V(E)$; $\alpha$ and $\beta$, and 2 terms for the energy dependence of the imaginary strengths; $\gamma_W$, and $\gamma_D$. Including the 4 free geometric parameters, $r_0$, $r_1$, $a_0$, and $a_1$, there are 15 free parameters total. 

